# Tutorial 2: Two component system

In tutorial 1 we became acquainted with how to build a database and start analyzing the properties of these systems. In tutorial 2 will look at a more complex system and check some of the conductivity calculations possible currently for two component materials.

In [ ]:
import mdsuite as mds
import os

Specifically, we will be looking at molten NaCl at 1400K. We start, as always, with the construction of the database under a new name, NaCl_1400K

In [ ]:
NaCl_1400K = mds.Trajectory(analysis_name="NaCl_1400K", 
                           new_project=True,
                          storage_path=".",
                          temperature=1400.0,
                          time_step=0.002,
                          time_unit=1e-12,
                          filename="trajectory_files/NaCl_1400K.dump",
                          length_unit=1e-10)

First, let's unwrap the coordinates in order to plot some msd characteristics

In [ ]:
NaCl_1400K.Unwrap_Coordinates()

The first point of dynamics analysis is usually the self diffusion, now of course, we have two components, so let's see how the program will handle this.

In [ ]:
NaCl_1400K.Einstein_Diffusion_Coefficients(plot=True)

If you only want to look at a single component, one simply needs to include the keyword argument as follows

In [ ]:
NaCl_1400K.Einstein_Diffusion_Coefficients(plot=True, species=["Na"])

So you can see that we will get only a single analysis performed.

We can also calculate the same property using the GK approach discussed in tutorial 1. Keep in mind that this simulation data is highly inappropriate for such a calculation.

In [ ]:
NaCl_1400K.Green_Kubo_Diffusion_Coefficients(plot=True, data_range=50)

So you can see that this value is much larger than what we would expect based on the Einstein approach taken above.

The other dynamic property of interest in many investigations it the ionic conductivity. For a fully uncorrelated system, this is trivially possible through the Nernst-Einstein relation, calculated in this code by simply running the Nernst-Einstein method. This method will automatically take care of deciding which implementations are possible. As it is not a computationally complicated calculation, there is not user override. 

In [ ]:
NaCl_1400K.Nernst_Einstein_Conductivity()

This approach however, neglects all correlation between the ionic species that may be present in the system. To account for this, we can run either the Einstein-Helfand analysis, or the Green-Kubo autocorrelation analysis to calculate this value. You must note that the simulation being used in this tutorial is very small and very short, therefore, the results that are to be shown will be inconsistent with each other, and with reality.

In [ ]:
NaCl_1400K.Einstein_Helfand_Conductivity(measurement_range=100, plot=True)

The error in this analysis comes by fitting a line over various ranges of the linear region and averaging over the calculated gradients

In [ ]:
NaCl_1400K.Green_Kubo_Conductivity(plot=True, data_range=150)

You can see in particular for the Green-Kubo that the data is highly inappropriate for this kind of analysis. Nonetheless, it can be performed. The issue with the trajectory is that it was sampled every 100 time steps, therefore, the GK approach will often fail as the resolution is too poor.